In [7]:
# Install packages manually if needed
#install_github(repo="ryantibs/best-subset", subdir="bestsubset", force=TRUE )
#install.packages("stabm")

In [8]:
# Load required libraries
library(devtools)
library(MASS)  # For mvrnorm
library(glmnet)  # For lasso and relaxed lasso
library(bestsubset)  # For best subset selection
library(leaps)  # For forward stepwise selection
library(stabm) # For Nogueira stability calculation

Warning message:
"package 'devtools' was built under R version 4.1.3"


ERROR: Error: package or namespace load failed for 'devtools' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'cli' 3.3.0 is already loaded, but >= 3.4.0 is required


In [9]:
tune_param = function(obj) {
  N = length(obj$err.test)  # Number of methods
  nrep = nrow(obj$err.test[[1]])  # Number of repetitions
  
  # Ensure N and nrep are numeric
  if (!is.numeric(N) || !is.numeric(nrep)) {
    stop("Number of methods (N) or repetitions (nrep) is not numeric. Check input structure.")
  }
  
  # Matrix for best validation tuning indices
  tun.val = matrix(NA, nrep, N)  
  tun.ora = rep(NA, N)           # Vector for best oracle tuning indices
  
  # Initialize a list to store selected features for validation tuning
  selected_features_val = vector(mode = "list", length = N)
  names(selected_features_val) <- names(obj$err.test)
  
  # Validation tuning: based on validation error
  for (i in 1:nrep) {
    for (j in 1:N) {
      # Find the best tuning parameter (minimum validation error)
      if (!is.null(obj$err.val[[j]][i, ])) {
        tun.val[i, j] = which.min(obj$err.val[[j]][i, ])
        
        if (!is.na(tun.val[i, j])) {
          # Extract selected features for the best tuning parameter
          selected_features_val[[j]][[i]] <- which(obj$selected_features[[j]][[i]] == 1)
        }
      }
    }
  }

  # Oracle tuning: based on average test error
  for (j in 1:N) {
    if (!is.null(obj$err.test[[j]])) {
      tun.ora[j] = which.min(colMeans(obj$err.test[[j]], na.rm = TRUE))
    }
  }
  
  # Add selected features for validation tuning to the output
  obj$tun.val <- tun.val
  obj$tun.ora <- tun.ora
  obj$selected_features_val <- selected_features_val
  
  return(obj)
}


In [10]:
master = function(n, p, nval, reg.funs, nrep = 50, seed = NULL, verbose = FALSE,
                  file = NULL, file.rep = 5, rho = 0, s = 5, beta.type = 1, snr = 1) {
  
  this.call = match.call()
  if (!is.null(seed)) set.seed(seed)

  N = length(reg.funs)
  reg.names = names(reg.funs)
  if (is.null(reg.names)) reg.names = paste("Method", 1:N)

  # Initialize lists
  err.train = err.val = err.test = prop = risk = nzs = fpos = fneg = F1 = 
    opt = runtime = selected_features = vector(mode = "list", length = N)
  names(err.train) = names(err.val) = names(err.test) = names(prop) =
    names(risk) = names(nzs) = names(fpos) = names(fneg) = names(F1) = 
    names(opt) = names(runtime) = names(selected_features) = reg.names
  
  stability = vector(mode = "numeric", length = N)  # For stability measures
  
  filled = rep(FALSE, N)
  err.null = risk.null = sigma = rep(NA, nrep)

  # Loop through the repetitions
  for (i in 1:nrep) {
    if (verbose) {
      cat(sprintf("Simulation %i (of %i) ...\n", i, nrep))
      cat("  Generating data ...\n")
    }

    # Generate x, y, xval, yval
    xy.obj = sim.xy(n, p, nval, rho, s, beta.type, snr)
    risk.null[i] = diag(t(xy.obj$beta) %*% xy.obj$Sigma %*% xy.obj$beta)
    err.null[i] = risk.null[i] + xy.obj$sigma^2
    sigma[i] = xy.obj$sigma

    # Loop through the regression methods
    for (j in 1:N) {
      if (verbose) {
        cat(sprintf("  Applying regression method %i (of %i) ...\n", j, N))
      }

      tryCatch({
        # Apply the regression method
        runtime[[j]][i] = system.time({
          reg.obj = reg.funs[[j]](xy.obj$x, xy.obj$y)
        })[1]

        # Grab the estimated coefficients and predictions
        if (names(reg.funs)[j] %in% c("ElasticNet", "AdaptiveLasso")) {
          # Handle Elastic Net and Adaptive Lasso
          betahat_full <- as.matrix(coef(reg.obj, s = "lambda.min"))
          betahat <- betahat_full[-1, , drop = FALSE]  # Drop intercept
          
          # Predictions
          muhat.train <- as.matrix(predict(reg.obj, newx = xy.obj$x, s = "lambda.min"))
          muhat.val <- as.matrix(predict(reg.obj, newx = xy.obj$xval, s = "lambda.min"))
        } else {
          # Handle other methods
          betahat = as.matrix(coef(reg.obj))
          m = ncol(betahat); nc = nrow(betahat)

          # Check for intercept
          if (nc == p + 1) {
            intercept = TRUE
            betahat0 = betahat[1, ]
            betahat = betahat[-1, ]
          } else intercept = FALSE

          muhat.train = as.matrix(predict(reg.obj, xy.obj$x))
          muhat.val = as.matrix(predict(reg.obj, xy.obj$xval))
        }
        m = ncol(betahat); nc = nrow(betahat)

        # Populate empty matrices for metrics
        if (!filled[j]) {
          err.train[[j]] = err.val[[j]] = err.test[[j]] = prop[[j]] =
            risk[[j]] = nzs[[j]] = fpos[[j]] = fneg[[j]] = F1[[j]] = opt[[j]] =
            matrix(NA, nrep, m)
          selected_features[[j]] = vector(mode = "list", length = nrep)
          filled[j] = TRUE
        }

        # Record all metrics
        err.train[[j]][i, ] = colMeans((muhat.train - xy.obj$y)^2)
        err.val[[j]][i, ] = colMeans((muhat.val - xy.obj$yval)^2)
        delta = betahat - xy.obj$beta
        risk[[j]][i, ] = diag(t(delta) %*% xy.obj$Sigma %*% delta)
        if (intercept) risk[[j]][i, ] = risk[[j]][i, ] + betahat0^2
        err.test[[j]][i, ] = risk[[j]][i, ] + xy.obj$sigma^2
        prop[[j]][i, ] = 1 - err.test[[j]][i, ] / err.null[i]
        nzs[[j]][i, ] = colSums(betahat != 0)
        tpos = colSums((betahat != 0) * (xy.obj$beta != 0))
        fpos[[j]][i, ] = nzs[[j]][i, ] - tpos
        fneg[[j]][i, ] = colSums((betahat == 0) * (xy.obj$beta != 0))
        F1[[j]][i, ] = 2 * tpos / (2 * tpos + fpos[[j]][i, ] + fneg[[j]][i, ])
        opt[[j]][i, ] = (err.test[[j]][i, ] - err.train[[j]][i, ]) / err.train[[j]][i, ]

        # Record selected features for the best tuning parameter
        best_tuning_idx <- which.min(err.val[[j]][i, ])
        selected_features[[j]][[i]] <- which(betahat[, best_tuning_idx] != 0)

      }, error = function(err) {
        if (verbose) {
          cat("    Oops! Something went wrong, see error message below; recording all metrics here as NAs ...\n")
          cat("    ***** Error message *****\n")
          cat(sprintf("    %s\n", err$message))
          cat("    *** End error message ***\n")
        }
      })
    }
  }

  stability <- list()

# Calculate stability measures for each method
  for (j in 1:N) {
  # Convert selected features to binary matrix
    binary_matrix <- matrix(0, nrow = length(selected_features[[j]]), ncol = p)
    for (i in seq_along(selected_features[[j]])) {
      selected <- selected_features[[j]][[i]]
      if (!is.null(selected)) {
        binary_matrix[i, selected] <- 1
      }
    }
  
  # Convert to list of selected indices for stabm
    selected_list <- lapply(1:nrow(binary_matrix), function(i) which(binary_matrix[i, ] == 1))
  
  # Compute Nogueira stability
    stability[[reg.names[j]]] <- stabilityNogueira(selected_list, p = p)
  }

  # Save final results
  out = list(
    err.train = err.train,
    err.val = err.val,
    err.test = err.test,
    err.null = err.null,
    prop = prop,
    risk = risk,
    risk.null = risk.null,
    nzs = nzs,
    fpos = fpos,
    fneg = fneg,
    F1 = F1,
    opt = opt,
    sigma = sigma,
    runtime = runtime,
    selected_features = selected_features,
    stability = stability
  )

  # Tune parameters based on validation error
  out <- tune_param(out)

  if (!is.null(file)) {
    saveRDS(out, file)
    invisible(out)
  } else {
    return(out)
  }
}


In [11]:
set.seed(123)

# Define regression methods
# Define regression functions
reg.funs <- list(
  Lasso = function(x, y) lasso(x, y, intercept = FALSE, nlam = 50),
  Stepwise = function(x, y) fs(x, y, intercept = FALSE),
  RelaxedLasso = function(x, y) lasso(x, y, intercept = FALSE, nlam = 50, nrelax = 5),
  ElasticNet = function(x, y) {
    # Elastic Net with alpha = 0.5 (equal mix of Lasso and Ridge penalties)
    fit <- cv.glmnet(x, y, alpha = 0.5, nfolds = 5)
    return(fit)
  },
  AdaptiveLasso = function(x, y) {
    # Fit Ridge regression to get initial weights
    ridge_fit <- glmnet(x, y, alpha = 0)
  
  # Extract coefficients for the best Ridge lambda (lambda.min)
    ridge_coef <- coef(ridge_fit, s = min(ridge_fit$lambda))[-1]  # Drop intercept
  
  # Compute adaptive weights
    weights <- 1 / (abs(as.numeric(ridge_coef)) + 1e-6)  # Avoid division by zero
  
  # Fit Lasso with penalty factors using cross-validation
    fit <- cv.glmnet(x, y, alpha = 1, penalty.factor = weights, nfolds = 5)
    return(fit)
  }
)

# Run the simulation
sim_results <- master(
  n = 100, p = 20, nval = 100, reg.funs = reg.funs,
  nrep = 10, rho = 0.5, s = 5, beta.type = 2, snr = 1, verbose = TRUE
)

Simulation 1 (of 10) ...
  Generating data ...


ERROR: Error in sim.xy(n, p, nval, rho, s, beta.type, snr): could not find function "sim.xy"


In [ ]:
sim_results

22.42138,20.98734,19.65509,18.40243,17.19183,16.13495,15.32253,14.676555,14.075450,13.503700,⋯,10.484304,10.483721,10.483278,10.482956,10.482708,10.482511,10.482357,10.482245,10.482157,10.482081
26.81964,24.05138,21.38819,19.03886,17.11377,15.61213,14.49229,13.661791,13.046098,12.589683,⋯,9.875129,9.873915,9.873035,9.872386,9.871871,9.871481,9.871217,9.871014,9.870859,9.870758
18.73832,17.61320,16.57219,15.57279,14.56730,13.64532,12.92473,12.360978,11.897345,11.545480,⋯,9.270304,9.269530,9.268942,9.268495,9.268162,9.267915,9.267718,9.267567,9.267462,9.267395
29.30666,25.42355,21.98388,19.23321,17.17839,15.64322,14.49626,13.639362,12.999192,12.520981,⋯,9.798318,9.797095,9.796216,9.795546,9.795022,9.794632,9.794309,9.794059,9.793867,9.793723
22.21629,19.90380,17.30936,15.06951,13.39366,12.12896,11.11235,10.320746,9.716436,9.265040,⋯,6.894779,6.893899,6.893265,6.892783,6.892413,6.892156,6.891945,6.891781,6.891656,6.891562
24.66880,23.18236,21.66724,20.18415,18.78082,17.69202,16.84514,16.157776,15.528928,15.051695,⋯,12.068322,12.067487,12.066851,12.066368,12.066014,12.065755,12.065541,12.065375,12.065256,12.065174
17.73917,15.98804,14.32238,12.90392,11.79125,10.93715,10.15001,9.531553,9.065556,8.717287,⋯,5.356405,5.355709,5.355206,5.354823,5.354524,5.354306,5.354130,5.353995,5.353898,5.353834
20.29150,18.56672,16.86311,15.37321,14.21336,13.27816,12.47570,11.801541,11.281000,10.886094,⋯,8.010815,8.009940,8.009300,8.008792,8.008401,8.008103,8.007878,8.007716,8.007600,8.007516
22.92706,20.96591,19.08271,17.49495,16.26125,15.31311,14.50336,13.882324,13.418383,13.071768,⋯,10.366971,10.366372,10.365904,10.365547,10.365282,10.365100,10.364950,10.364835,10.364751,10.364698
25.43913,23.02173,20.96881,19.21921,17.73419,16.52694,15.62486,14.939086,14.392994,13.926455,⋯,10.624678,10.623674,10.622958,10.622411,10.621982,10.621657,10.621437,10.621263,10.621131,10.621043
22.42138,16.47923,13.761112,12.968947,12.154766,11.693849,11.393186,11.121603,10.977615,10.899511,⋯,10.599686,10.563953,10.538180,10.515170,10.500021,10.490529,10.482988,10.481887,10.481808,10.481808


In [ ]:
sim_results$stability

$Lasso
[1] 0.2565883

$Stepwise
[1] 0.4860245

$RelaxedLasso
[1] 0.701846

$ElasticNet
[1] 0.4024813

$AdaptiveLasso
[1] 0.3397472